In [1]:
import glob
import pandas as pd
import cv2
import numpy as np
import matplotlib.image as mpimg
import os
import tensorflow as tf

/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [2]:
# 打开数据集
filelist = []
for file in glob.glob(r'data/classification-task/*/test.csv'):   #csv所在路径
    df = pd.read_csv(file)
    filelist.append(df)
df = pd.concat(filelist)
X = []
y = []
for i in df.itertuples():
    I = mpimg.imread(i.path.replace('../../', 'data/'))
    X.append(I)
    y.append(i.latex)
df = pd.get_dummies(y)
column_name = df.columns.to_list()
test_x, test_y = np.array(X), np.array(pd.get_dummies(y))
test_x = test_x / 255

In [3]:
def find_near_four_pixels(row, col, height, width):
    if row == 0 and col == 0:
        neighbours = None
    elif row == 0:
        neighbours = [(row, col - 1)]
    elif col == 0:
        neighbours = [(row - 1, col), (row - 1, col + 1)]
    elif col == width - 1:
        neighbours = [(row - 1, col), (row - 1, col - 1), (row, col - 1)]
    else:
        neighbours = [(row - 1, col), (row - 1, col - 1), (row - 1, col + 1), (row, col - 1)]
    return neighbours

In [4]:
def data_label():
    filelist = []
    for file in glob.glob(r'data/classification-task/*/test.csv'):  #   csv文件所在路径   train/test
        df = pd.read_csv(file)
        filelist.append(df)
    df = pd.concat(filelist)
    df.drop_duplicates(['symbol_id'], inplace=True)

    old_name = list(df['latex'])
    old_label = list(df['symbol_id'])
    index = list(range(369))
    name_dict = dict(zip(old_label, old_name))
    # label_dict = dict(map(reversed, enumerate(old_label)))
    label_dict = dict(zip(index, old_label))
    return label_dict, name_dict

In [5]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            graph_def,
            input_map=None,
            return_elements=None,
            name="prefix",
            op_dict=None,
            producer_op_list=None
        )
    return graph

In [6]:
def model_predict(filepath, sess, x, y, z):
    label_dict, name_dict = data_label()

    # input = cv2.imread(filepath)
    image = cv2.resize(filepath, (32, 32))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    image = np.expand_dims(image, axis=0)
    y_out = sess.run(y, feed_dict={
        x: image,
        z: 1
    })
    output = name_dict.get(label_dict.get(np.argmax(y_out, 1)[0]))
    print(output)
    return output

In [7]:
def cropimage(file):
    os.system('rm -rf result')
    os.system('mkdir result')
    img = cv2.imread(file)
    result = img.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)[1]
    height, width = thresh.shape
    linked = {}
    labels = np.zeros((height, width))

    row = 0
    next_label = 1
    big_array = []
    while row < height:
        col = 0
        while col < width:
            # 如果现在的点是黑色
            if thresh[row][col] == 0:
                neighbours = find_near_four_pixels(row, col, height, width)
                # 删除背景像素（白色)
                temp = []
                for item in neighbours:
                    if thresh[item[0]][item[1]] == 0:
                        neighbours.remove(item)
                        temp.append(item)
                # print(row,col)
                if temp == []:
                    linked[next_label] = [(row, col)]
                    labels[row][col] = next_label
                    next_label += 1
                else:
                    mini = 100000
                    coord = None
                    for item in temp:
                        if labels[item[0]][item[1]] < mini:
                            mini = labels[item[0]][item[1]]
                            coord = item
                    if coord is not None:
                        labels[row][col] = mini
                    # print(neighbours,row,col,height,width)
                    linked[mini].append((row, col))
                    for item in temp:
                        label = labels[item[0]][item[1]]
                        if label != mini:
                            linked[mini] += linked[label]
                            if linked[mini] in big_array:
                                big_array.remove(linked[mini])
                            if linked[mini] not in big_array:
                                big_array.append(linked[mini])
                            if linked[label] in big_array:
                                big_array.remove(linked[label])
            col += 1
        row += 1

    graph = load_graph(os.path.join('./logs', 'frozen_model2.pb'))     #pb文件所在路径
    x = graph.get_tensor_by_name('prefix/Placeholder:0')
    y = graph.get_tensor_by_name('prefix/output_node:0')
    z = graph.get_tensor_by_name('prefix/training:0')
    with tf.Session(graph=graph) as sess:
        # key = [i for i in linked]
        i = 0
        f = open("output.txt", "w")
        for component in big_array:
            max_row = 0
            min_row = height
            max_col = 0
            min_col = width
            # print(i+1)
            for item in component:
                if max_row <= item[0]:
                    max_row = item[0]
                if min_row >= item[0]:
                    min_row = item[0]
                if max_col <= item[1]:
                    max_col = item[1]
                if min_col >= item[1]:
                    min_col = item[1]
            temp_width = 1 + max_col - min_col
            temp_height = 1 + max_row - min_row
            outputimage = thresh[min_row - 2:max_row + 2, min_col - 2:max_col + 2]
            area = temp_width * temp_height
            if area >= 20:
                outputimage = cv2.resize(outputimage, (32, 32), interpolation=cv2.INTER_AREA)
                cv2.imwrite("./result/" + str(i) + ".png", outputimage)

                pic = cv2.imread("./result/" + str(i) + ".png")
                (T, pic) = cv2.threshold(pic, 155, 255, cv2.THRESH_BINARY)
                pic = cv2.cvtColor(pic, cv2.COLOR_BGR2RGB)
                
                character = model_predict(pic, sess, x, y, z)
                output = str(character) + " (" + str(min_row - 2) + str(min_col - 2) + str(max_row + 2) + str(
                    max_col + 2) + ") " + "./result/" + str(i) + ".png"
                f.write(output + '\n')
                i += 1

    labels[labels == 0] = -1
    labels[labels > -1] = 0
    labels[labels == -1] = 255
    cv2.imwrite('graph.png', labels)
    f.close()
    return labels

In [8]:
cropimage('./test/1.png')

Instructions for updating:
Please file an issue at https://github.com/tensorflow/tensorflow/issues if you depend on this feature.
\mathscr{H}
\mathscr{H}
\Omega
\mathfrak{S}
\blacksquare
\mathfrak{S}
x
\nexists
2
\mathfrak{A}
\mathfrak{A}
\aa
n
n
V
\neq
\twoheadrightarrow
\leqslant
\mathfrak{S}
u
E
\mathfrak{S}
\alpha
8
\cap
\leqslant
8
\triangleq
V
\Omega
\mathcal{L}
\zeta
\nsubseteq
\mathscr{H}
\mathfrak{A}
\mathfrak{A}
8
8
\mathfrak{A}
\mathfrak{A}
\mathfrak{A}
\fullmoon
V
\sigma
\mathscr{H}
L
\sphericalangle
\sigma
y
\int
L
P
p
6
\mathfrak{S}
\mathfrak{S}
\between
\alpha
M
\hookrightarrow
\hookrightarrow
\S
\geqslant
\mathcal{U}
n
\alpha
8
\sigma
\AE
\mathbb{H}
V
T
\nsubseteq
\sigma
\mathbb{H}
3
\sigma
\mathscr{F}
L
\Lambda
\leqslant
\mathcal{M}
\mathfrak{A}
[
x
\mars
\mathcal{X}
8
8
\mathcal{P}
\mathscr{A}
P
3
V
\mathfrak{M}
\diamondsuit
L
\mathfrak{X}
\diameter
X
\mathcal{X}
V
\mathbb{H}
\alpha
T
8
\mathscr{A}
V
P
V
\leqslant
P
\mathcal{O}
\rightharpoonup
\clubsuit
8
\mathcal{M}


array([[255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.],
       ...,
       [255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.]])